<a href="https://colab.research.google.com/github/samipn/Pycaret/blob/main/01_binary_classification_banknote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Classification (Banknote Authentication) — PyCaret

# ✅ Enable GPU in Colab
*Runtime → Change runtime type → **T4 / L4 GPU** → Save.*  
Each notebook sets `use_gpu=True` in `setup()`. Models that support GPU (e.g., XGBoost, CatBoost) will leverage it automatically if available.

Dataset from UCI via public mirror. Target: `class` (0/1).

In [5]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 8.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of umap-learn to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dash[testing] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dash[testing] to determine which version is compatible with other requirements. This could take a while.
INF

> **Optional Kaggle download**
> Uncomment the Kaggle cell and add your *kaggle.json* (Account → Create New API Token).  
> Then run the cell to fetch the dataset from Kaggle instead of the default public CSV.

In [ ]:
# If you want Kaggle instead, uncomment and configure:
# !pip -q install kaggle
# import os, json
# os.makedirs('/root/.kaggle', exist_ok=True)
# # Upload kaggle.json via Colab's file pane and place it in /root/.kaggle
# os.chmod('/root/.kaggle/kaggle.json', 0o600)
# !kaggle datasets download -d jpchen1228/uci-banknote-authentication
# !unzip -o uci-banknote-authentication.zip -d data
# DATA_PATH = 'data/data_banknote_authentication.txt'

In [1]:
import pandas as pd

# Public CSV mirror (Jason Brownlee Datasets)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/banknote_authentication.csv"
cols = ["variance","skewness","curtosis","entropy","class"]
df = pd.read_csv(url, names=cols)

print(df.head())
print(df['class'].value_counts())

   variance  skewness  curtosis  entropy  class
0   3.62160    8.6661   -2.8073 -0.44699      0
1   4.54590    8.1674   -2.4586 -1.46210      0
2   3.86600   -2.6383    1.9242  0.10645      0
3   3.45660    9.5228   -4.0112 -3.59440      0
4   0.32924   -4.4552    4.5718 -0.98880      0
class
0    762
1    610
Name: count, dtype: int64


In [3]:
from pycaret.classification import setup, compare_models, pull, create_model, tune_model, blend_models, stack_models, finalize_model, predict_model, save_model

s = setup(
    data=df,
    target='class',
    session_id=42,
    use_gpu=True,
    normalize=True,
    train_size=0.8,
    fix_imbalance=True,
    fold=5
)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split req

,Description,Value
0,Session id,42
1,Target,class
2,Target type,Binary
3,Original data shape,"(1372, 5)"
4,Transformed data shape,"(1493, 5)"
5,Transformed train set shape,"(1218, 5)"
6,Transformed test set shape,"(275, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split req

In [4]:
best = compare_models(n_select=3)
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9982,0.9984,1.0000,0.9959,0.9980,0.9963,0.9963,0.5920
et,Extra Trees Classifier,0.9982,1.0000,1.0000,0.9959,0.9980,0.9963,0.9963,0.2040
catboost,CatBoost Classifier,0.9964,0.9999,1.0000,0.9919,0.9959,0.9926,0.9927,19.5640
xgboost,Extreme Gradient Boosting,0.9945,0.9998,0.9938,0.9938,0.9938,0.9889,0.9889,0.1520
gbc,Gradient Boosting Classifier,0.9936,0.9998,0.9939,0.9919,0.9928,0.9871,0.9871,0.2600
lightgbm,Light Gradient Boosting Machine,0.9936,0.9997,0.9959,0.9900,0.9929,0.9871,0.9872,1.2000
ada,Ada Boost Classifier,0.9927,0.9980,0.9959,0.9879,0.9919,0.9852,0.9853,0.1380
qda,Quadratic Discriminant Analysis,0.9881,1.0000,1.0000,0.9742,0.9869,0.9761,0.9764,0.0300
dt,Decision Tree Classifier,0.9863,0.9867,0.9897,0.9798,0.9847,0.9723,0.9724,0.0320
lr,Logistic Regression,0.9845,0.9997,0.9959,0.9702,0.9829,0.9687,0.9691,1.2500


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

[2025-10-26 19:53:53.202] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


[2025-10-26 19:56:05.210] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:05.210] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9982,0.9984,1.0000,0.9959,0.9980,0.9963,0.9963,0.592
et,Extra Trees Classifier,0.9982,1.0000,1.0000,0.9959,0.9980,0.9963,0.9963,0.204
catboost,CatBoost Classifier,0.9964,0.9999,1.0000,0.9919,0.9959,0.9926,0.9927,19.564
xgboost,Extreme Gradient Boosting,0.9945,0.9998,0.9938,0.9938,0.9938,0.9889,0.9889,0.152
gbc,Gradient Boosting Classifier,0.9936,0.9998,0.9939,0.9919,0.9928,0.9871,0.9871,0.260
lightgbm,Light Gradient Boosting Machine,0.9936,0.9997,0.9959,0.9900,0.9929,0.9871,0.9872,1.200
ada,Ada Boost Classifier,0.9927,0.9980,0.9959,0.9879,0.9919,0.9852,0.9853,0.138
qda,Quadratic Discriminant Analysis,0.9881,1.0000,1.0000,0.9742,0.9869,0.9761,0.9764,0.030
dt,Decision Tree Classifier,0.9863,0.9867,0.9897,0.9798,0.9847,0.9723,0.9724,0.032
lr,Logistic Regression,0.9845,0.9997,0.9959,0.9702,0.9829,0.9687,0.9691,1.250


In [5]:
# Try tuning and blending
tuned = [tune_model(m) for m in best]
blended = blend_models(estimator_list=tuned)
stacked = stack_models(estimator_list=tuned, meta_model=None)
final_best = finalize_model(blended)

print(final_best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9955,0.9959,1.0000,0.9899,0.9949,0.9908,0.9909
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9954,1.0000,1.0000,0.9899,0.9949,0.9908,0.9908
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9909,0.9959,1.0000,0.9798,0.9898,0.9815,0.9817
Mean,0.9964,0.9984,1.0000,0.9919,0.9959,0.9926,0.9927
Std,0.0034,0.0020,0.0000,0.0076,0.0038,0.0069,0.0068


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[2025-10-26 19:56:34.115] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:34.434] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[2025-10-26 19:56:34.437] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:34.452] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:35.579] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:35.597] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:35.607] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:35.621] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:35.631] [CUML] [info] Unused keyword para

[2025-10-26 19:56:37.423] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:56:37.424] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9955,1.0000,1.0000,0.9899,0.9949,0.9908,0.9909
1,0.9864,1.0000,1.0000,0.9703,0.9849,0.9725,0.9729
2,0.9817,0.9998,1.0000,0.9608,0.9800,0.9632,0.9639
3,0.9909,0.9998,1.0000,0.9798,0.9898,0.9815,0.9817
4,0.9863,0.9999,1.0000,0.9700,0.9848,0.9723,0.9727
Mean,0.9881,0.9999,1.0000,0.9742,0.9869,0.9761,0.9764
Std,0.0047,0.0001,0.0000,0.0099,0.0051,0.0094,0.0092


...

[2025-10-26 19:59:42.097] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:59:42.098] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9954,1.0000,1.0000,0.9898,0.9949,0.9908,0.9908
Mean,0.9991,1.0000,1.0000,0.9980,0.9990,0.9982,0.9982
Std,0.0018,0.0000,0.0000,0.0041,0.0021,0.0037,0.0037


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

[2025-10-26 19:59:45.690] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 19:59:45.691] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


[2025-10-26 20:00:09.074] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-10-26 20:00:09.075] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['variance', 'skewness', 'curtosis',
                                             'entropy'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              strategy='mean'))),
                ('categoric

In [6]:
# Evaluate on hold-out
preds = predict_model(final_best)
preds.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,variance,skewness,curtosis,entropy,class,prediction_label,prediction_score
385,2.37180,7.49080,0.015989,-1.74140,0,0,1.0000
899,-1.44460,2.14380,-0.472410,-1.66770,1,1,0.9981
47,-0.78690,9.56630,-3.786700,-7.50340,0,0,0.9892
1018,-0.40804,0.54214,-0.527250,0.65860,1,1,0.9998
786,-2.09620,-7.10590,6.618800,-0.33708,1,1,0.9987


In [7]:
# Save the best model (PyCaret 3.x) and export metrics
best_path = save_model(best, 'best_model')
print("Saved model:", best_path)

# Export experiment logs / results
import pandas as pd
results_df = pull()  # last displayed table
results_df.to_csv('experiment_results.csv', index=False)
print("Exported experiment results to experiment_results.csv")

Transformation Pipeline and Model Successfully Saved
Saved model: (Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['variance', 'skewness', 'curtosis',
                                             'entropy'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              strategy='mean'))),
                ('categorical_imputer',
                 TransformerWrapper(exclude=None, include=...
                                       class_weight=None, criterion='gini',
  